# Tutorial de exemplo ZODB - Python 2.7.13

In [1]:
from ZODB import FileStorage, DB

In [2]:
#Criando ou chamando o arquivo 'meusdados.fs' como um sistema de arquivos para a variavel storage

storage = FileStorage.FileStorage('meusdados.fs')

In [3]:
#Criando um banco de dados que utiliza o storage

db = DB(storage=storage)

É aberta uma conexão com o banco de dados pelo metodo open(). O retorno será um objeto do tipo connection para o banco de dados. 
Esse objeto lhe dará acesso 'root' se usada a função root()

In [4]:

connection = db.open()

In [5]:
root = connection.root()

O objeto Root é um dicionário ( como se fosse um set do java - um par chave : valor - que terá todas as persistências dos seus objetos. Como um exemplo, você pode armazenar listas de strings no objeto root:

In [7]:
#root['alunos'] = ['Vinicius', 'Lucas','Alex']
root['pedidos'] = ['1', '2', '3']

Com essa inserção, temos apenas uma mudança temporária. Para torná-la permanente devemos commitar a transação atual:

In [13]:
import transaction

In [14]:
transaction.commit()

# Sobre transações

É possível compreender transações como 'checkpoints' onde se salva as alterações feitas nos objetos até o momento. Vamos agora descobrir se nossos dados foram salvos. Primeiro vamos fechar a conexão com o banco de dados.

In [10]:
connection.close()

Reinicie o Kernel do Python. Faça os imports novamente. Vamos agora reabrir a conexão com o banco de dados que nós criamos 

In [2]:
storage = FileStorage.FileStorage('meusdados.fs')

In [3]:
db = DB(storage)

In [4]:
connection = db.open()

In [5]:
root = connection.root()

Agora podemos ver o que está armazenado no root:

In [6]:
root.items()

[('alunos', ['Vinicius', 'Lucas', 'Alex']),
 ('accounts', <BTrees.OOBTree.OOBTree at 0x7f807aebcb00>),
 ('pedidos', ['1', '2', '3'])]


Ali está nossa lista de alunos. Se tivessemos usado um banco relacional, nós teríamos que fazer uma query SQL para salvar até mesmo um exemplo simples como esse. 
Seria necessário que um código Python que convertesse o SQL de volta na lista quando você tivesse interesse de usá-la. 
Nesse sentido não é preciso fazer nada parecido quando usamos o ZODB. O uso do ZODB é quase totalmente transparente, de fato, programas baseados em ZODB funcionam de forma muito simples.

# Detectando Mudanças

No ZODB não precisamos manter os olhos atentos para as mudanças nas variáveis e seus estados. Tudo que precisamos é fazer as mudanças na persistencia dos objetos e fazer o commit da transação. Tudo que for alterado será armazenado no banco.

A única exceção para essa regra é quando se trata de itens mutáveis do Python como listas e dicionários. Se você alterar uma lista ou dicionário que já está no banco, nesse caso a mudança não terá efeito. Por exemplo:

In [7]:
root['alunos'].append('Cleber')

In [8]:
transaction.commit()

NameError: name 'transaction' is not defined

Normalmente se espearria que isso funcionasse, mas não funciona. A razão é que o ZODB não detecta que a lista 'alunos' alterou. A lista 'alunos' é um objeto mutável que não notifica o ZODB quando sofre alterações.

Existe algumas maneiras de contornar esse problema. Um dos jeitos é reatribuir o objeto alterado:

In [9]:
l_alunos = root['alunos']

In [10]:
l_alunos.append('Cleber')

In [11]:
root['alunos'] = l_alunos

In [12]:
transaction.commit()

NameError: name 'transaction' is not defined

Aqui movemos a lista de alunos para uma variável local, alteramos a lista e somente então reatribuímos devolta para o banco de dados e comitamos a transação. Esssa reatribuição notifica o bnaco de dados de que a lista mudou e que precisa ser salva no banco de dados.

# Usando classes - Persistent Classes

A forma mais fácil de criar objetos mutáveis que notificam o ZODB de mudanças é criar uma classe persistente. Classes persistentes permitem que você armazene o seu próprio tipo de objetos dentro do banco. Por exemplo, vamos considerar a classe aluno:

In [15]:
import persistent

In [16]:
class Aluno(persistent.Persistent):
    
    def setName(self,name):
        self.name = name        

Para criar uma classe persistente, basta adicionar a herança da classe persistent.Persistent. Agora podemos por objetos alunos no banco de dados:

In [17]:
alunos = []

In [18]:
for name in ['Renato', 'Maria', 'Douglas']:
    aluno = Aluno()
    aluno.setName(name)
    alunos.append(aluno)    

In [19]:
alunos
for aluno in alunos:
    print aluno.name

Renato
Maria
Douglas


In [20]:
root['alunos'] = alunos

In [21]:
transaction.commit()

In [23]:
for aluno in root['alunos']:
    print aluno.name

Renato
Maria
Douglas


Não esquecer de chamar commit(), e então as mudanças que foram feitas até aquele momento sejam enviadas para o banco de dados, e então uma nova transação começa.
Com a nova estrutura, agora podemos fazer mudanças na lista alunos salva no banco. Por exemplo, vamos mudar o nome de Renato para Vinícius:

In [24]:
renato = root['alunos'][0]

In [25]:
renato.name

'Renato'

In [26]:
renato.setName('Vinicius')

In [27]:
transaction.commit()

In [28]:
l_alunos = root['alunos']

In [29]:
for aluno in alunos:
    print aluno.name

Vinicius
Maria
Douglas


Como podemos ver a alteração foi feita!